<a href="https://colab.research.google.com/github/chandankumar3it/bank-loan-eda-analysis/blob/main/Bank_Loan_Case_Study_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Required Libraries

In [1]:
#Importing all the important libraries like numpy. pandas, matlplolib, and warnings to keep notebook clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# To suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
#notebook setting to display all the rowns and columns to have better clearity on the data.

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

# Work on Dataset - application_data.csv

## Reading and Understanding the Dataset

#### Importing the dataset





In [135]:
# importing application_data.csv

appData_df = pd.read_csv("application_data.csv")

#### Understanding the dataset

In [ ]:
appData_df.head()

In [137]:
#Checking the rows and columns of the raw dataset

appData_df.shape

(307511, 122)

In [ ]:
#Checking information of all the columns like data types
appData_df.info("all")

In [ ]:
# Checking the numeric variables of the dataframes
appData_df.describe()

**INSIGHT**


*   There are 122 columns and 307511 rows.
*   There columns having negative, postive values which includes days. It is required to fix.
*   There are columns with very hight values, columns related to Amount. Standardising is required









## Data Cleaning & Manipulation

### Data Quality Check - Missing Values

In [140]:
#checking how many null values are present in each of the columns

#creating a function to find null values for the dataframe
def missing_values(appData_df):
    return 100*appData_df.isnull().mean().sort_values(ascending = False)


In [ ]:
# Missing values columns

null_col = missing_values(appData_df)
null_col

**INSIGHT**


### Remove the columns with Missing values more than 40%

In [ ]:
#creating a variable missing_value_col for storing null columns having missing values more than 40%

missing_value_col_40 = null_col[null_col>40]
missing_value_col_40

In [ ]:
#Revieving missing_value_col

print(missing_value_col_40)
print()
print("Number of columns having missing values more than 40% :",len(missing_value_col_40))

**INSIGHT**

* There are 49 columns having null values more than 40% which are related to different area sizes on apartment owned/rented by the loan applicant

In [ ]:
# We will drop all these columns
missing_value_col_40.index

In [146]:
# Drop all the columns having missing values more than 40%

appData_df.drop(columns = missing_value_col_40.index, inplace = True)

In [147]:
appData_df.shape

(307511, 73)

*** After after dropping 49 columns we have left with 73 columns**

### Dealing with null values less than 15%

In [ ]:
# Columns with null values < 15%

missing_value_col_15 = null_col[null_col<15]
print("Number of columns with null value less than 15% :", len(missing_value_col_15.index))
print(missing_value_col_15)


*   There are 71 columns which have less than 15% missing values



In [ ]:
missing_value_col_15.index

In [ ]:
# Reviewing the columns
print(missing_value_col_15)
print()
print("Number of columns having missing values less than 15% :",len(missing_value_col_15))

### Analyse & Removing Unneccsary Columns

In [ ]:
# Identifying unique values with columns < 15%

appData_df[missing_value_col_15.index].nunique().sort_values(ascending=False)

* **From the above we can see that first two (EXT_SOURCE_2, AMT_GOODS_PRICE) are continous variables and remaining are catagorical variables**

In [ ]:
# Continous varibale - EXT_SOURCE_2

sns.boxplot(appData_df['EXT_SOURCE_2'])
plt.show()

In [ ]:
# Continous varibale - AMT_GOODS_PRICE

sns.boxplot(appData_df['AMT_GOODS_PRICE'])
plt.show()

Observation from Boxplots:
*   For 'EXT_SOURCE_2' no outliers present. So data is rightly present.
*   For 'AMT_GOODS_PRICE' outlier present in the data. So need to impute with median.



In [ ]:
for col in appData_df.columns:
    print(col)

### Removing the un-used columns and analysis

In [155]:
# Un-used columns in data set
unused_col = ['FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE','FLAG_PHONE', 'FLAG_EMAIL',
          'REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','FLAG_EMAIL','CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
          'REGION_RATING_CLIENT_W_CITY','FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3','FLAG_DOCUMENT_4',
          'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6','FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10',
          'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
          'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18','FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
          'FLAG_DOCUMENT_21','EXT_SOURCE_2','EXT_SOURCE_3']

In [156]:
# Droping un-used columns
appData_df.drop(labels = unused_col, axis=1, inplace = True)

In [ ]:
appData_df.head()

In [158]:
appData_df.shape

(307511, 42)

## Imputing values

In [159]:
# Imputing the value'XNA' which means not available for the column 'CODE_GENDER'

appData_df.CODE_GENDER.value_counts()

F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64

* XNA values are very low and Female is the majority. So lets replace XNA with gender 'F'

In [160]:
# Replacing 'XNA' to 'F' for 'CODE_GENDER'

appData_df.loc[appData_df.CODE_GENDER == 'XNA', 'CODE_GENDER'] = 'F'

In [161]:
# Reviewing the 'CODE_GENDER'
appData_df.CODE_GENDER.value_counts()

F    202452
M    105059
Name: CODE_GENDER, dtype: int64

In [ ]:
# checking the CODE_GENDER

appData_df.CODE_GENDER.head(10)

In [163]:
appData_df["CODE_GENDER"].isnull().sum()

0

### Imputing for "OCCUPATION_TYPE" column

In [ ]:
#Percentage of each category present in "OCCUPATION_TYPE"

appData_df["OCCUPATION_TYPE"].value_counts(normalize=True)*100

In [165]:
# Checking null value in column OCCUPATION_TYPE
appData_df["OCCUPATION_TYPE"].isnull().sum()

96391

* There are total 96391 records/rows having null value in columns OCCUPATION_TYPE

**Insight:**

* From above it looks like this columnn is categorical one and have missing values.
* To fix this we will impute another category as "Unknown" for the missing values.

In [166]:
# imputing null values with "Unknown"

appData_df["OCCUPATION_TYPE"] = appData_df["OCCUPATION_TYPE"].fillna("Unknown")

In [167]:
# Reviewing the null values in column OCCUPATION_TYPE
appData_df["OCCUPATION_TYPE"].isnull().sum()

0

In [ ]:
# Plotting a percentage graph having each category of "OCCUPATION_TYPE"

plt.figure(figsize = [12,7])
(appData_df["OCCUPATION_TYPE"].value_counts()).plot.barh(color= "orange",width = .8)
plt.title("Type of Occupations", fontdict={"fontsize":20}, pad =20)
plt.show()

* **Highest percentage of values belongs to Unknown group and Secons belongs to Laborers**



In [ ]:
appData_df.info("all")

### **Now let's move to other 6 columns :**
**"AMT_REQ_CREDIT_BUREAU_YEAR", "AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_MON", "AMT_REQ_CREDIT_BUREAU_WEEK","AMT_REQ_CREDIT_BUREAU_DAY", "AMT_REQ_CREDIT_BUREAU_HOUR"**

In [ ]:
appData_df[["AMT_REQ_CREDIT_BUREAU_YEAR","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_WEEK",
"AMT_REQ_CREDIT_BUREAU_DAY","AMT_REQ_CREDIT_BUREAU_HOUR"]].describe()

* **These above columns represent number of enquries made for the customer(which should be discrete and not continous).**
* **From above describe results we see that all values are numerical and can conclude that for imputing missing we should not use mean as it is in decimal form, hence for imputing purpose we will use median for all these columns.**

In [171]:
#creating "amt_credit" variable having these columns "AMT_REQ_CREDIT_BUREAU_YEAR","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_WEEK",
#"AMT_REQ_CREDIT_BUREAU_DAY","AMT_REQ_CREDIT_BUREAU_HOUR"

amt_req_credit = ["AMT_REQ_CREDIT_BUREAU_YEAR","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_WEEK",
"AMT_REQ_CREDIT_BUREAU_DAY","AMT_REQ_CREDIT_BUREAU_HOUR"]

In [172]:
#filling missing values with median values

appData_df.fillna(appData_df[amt_req_credit].median(),inplace = True)

In [173]:
missing_values(appData_df).head(10)

NAME_TYPE_SUITE                0.420148
OBS_30_CNT_SOCIAL_CIRCLE       0.332021
DEF_60_CNT_SOCIAL_CIRCLE       0.332021
OBS_60_CNT_SOCIAL_CIRCLE       0.332021
DEF_30_CNT_SOCIAL_CIRCLE       0.332021
AMT_GOODS_PRICE                0.090403
AMT_ANNUITY                    0.003902
DAYS_LAST_PHONE_CHANGE         0.000325
REG_REGION_NOT_WORK_REGION     0.000000
LIVE_REGION_NOT_WORK_REGION    0.000000
dtype: float64

**Still there some missing value coloumns but we will not impute them as the missing value count very less.**

In [174]:
# Casting variable into numeric in the dataset

numerical_columns=['TARGET','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','REGION_POPULATION_RELATIVE',
                 'DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH','HOUR_APPR_PROCESS_START',
                 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
                'DAYS_LAST_PHONE_CHANGE']

In [ ]:
appData_df[numerical_columns] = appData_df[numerical_columns].apply(pd.to_numeric)
appData_df.head(10)

In [176]:
appData_df.shape

(307511, 42)

## **Standardising values**

In [177]:
appData_df.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,1.422245,0.143421,1.405292,0.100049,-962.858788,0.005538,0.006055,0.029723,0.231293,0.229631,1.778463
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,3522.886321,1509.450419,3.265832,0.122126,0.219526,0.197499,0.268444,0.421124,0.383817,2.400989,0.446698,2.379803,0.362291,826.808487,0.078014,0.103037,0.190728,0.856810,0.744059,1.765523
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4292.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,-7479.500000,-4299.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1570.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,-4504.000000,-3254.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-757.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,-2010.000000,-1720.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,-274.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,0.000000,0.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,348.000000,34.000000,344.000000,24.000000,0.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


**Insights:**

From above describe result we can see that

* Columns DAYS_BIRTH, DAYS_EMPLOYED, DAYS_REGISTRATION, DAYS_ID_PUBLISH, DAYS_LAST_PHONE_CHANGE which counts days have negative values thus will correct those values
* Convert DAYS_BIRTH to AGE in years , DAYS_EMPLOYED to YEARS EMPLOYED
* Columns AMT_INCOME_TOTAL, AMT_CREDIT, AMT_GOODS_PRICE have very high values, thus will make these numerical columns in categorical columns for better understanding.



### **Dealing with columns :**
**"DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "DAYS_LAST_PHONE_CHANGE"**


**Columns DAYS_BIRTH, DAYS_EMPLOYED, DAYS_REGISTRATION, DAYS_ID_PUBLISH, DAYS_LAST_PHONE_CHANGE which counts days have negative values. thus will correct those values**


In [ ]:
# creating "days_col" varibale to store all days columns
days_col = ["DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "DAYS_LAST_PHONE_CHANGE"]

appData_df[days_col].describe()

**From above, we get that days are in negative that is not usual, so to correct it we use absolute function as below**

In [179]:
#using abs() function to correct the days values

appData_df[days_col]= abs(appData_df[days_col])

In [ ]:
# Reviewing correct data

appData_df[days_col].describe()

**Now convert DAYS_BIRTH, DAYS_EMPLOYED columns in terms of Years and binning years for better understanding, that is adding two more categorical column**

In [181]:
appData_df["AGE"] = appData_df["DAYS_BIRTH"]/365
bins = [0,20,25,30,35,40,45,50,55,60,100]
slots = ["0-20","20-25","25-30","30-35","35-40","40-45","45-50","50-55","55-60","60 Above"]

appData_df["AGE_GROUP"] = pd.cut(appData_df["AGE"], bins=bins, labels=slots)

In [ ]:
appData_df["AGE_GROUP"].value_counts(normalize= True)*100

In [183]:
#creating column "EMPLOYEMENT_YEARS" from "DAYS_EMPLOYED"

appData_df["YEARS_EMPLOYED"] = appData_df["DAYS_EMPLOYED"]/365
bins = [0,5,10,15,20,25,30,50]
slots = ["0-5","5-10","10-15","15-20","20-25","25-30","30 Above"]

appData_df["EMPLOYEMENT_YEARS"] = pd.cut(appData_df["YEARS_EMPLOYED"], bins=bins, labels=slots)

In [ ]:
appData_df["EMPLOYEMENT_YEARS"].value_counts(normalize= True)*100

**Taking care of Columns: AMT_INCOME_TOTAL, AMT_CREDIT, AMT_GOODS_PRICE**

In [185]:
# Binning Numerical Columns to create a categorical column

# Creating bins for AMT_INCOME_TOTAL in term of Lakhs
appData_df['AMT_INCOME_TOTAL']=appData_df['AMT_INCOME_TOTAL']/100000

bins = [0,1,2,3,4,5,6,7,8,9,10,100]
slot = ['0-1L','1L-2L', '2L-3L','3L-4L','4L-5L','5L-6L','6L-7L','7L-8L','8L-9L','9L-10L','10L Above']

appData_df['AMT_INCOME_RANGE']=pd.cut(appData_df['AMT_INCOME_TOTAL'],bins,labels=slot)

In [186]:
appData_df["AMT_INCOME_RANGE"].value_counts(normalize = True)*100

1L-2L        50.697218
2L-3L        21.194896
0-1L         20.714258
3L-4L         4.772559
4L-5L         1.743369
5L-6L         0.356088
6L-7L         0.282594
8L-9L         0.096908
10L Above     0.080323
7L-8L         0.052682
9L-10L        0.009105
Name: AMT_INCOME_RANGE, dtype: float64

In [187]:
# Creating bins for AMT_CREDIT in term of Lakhs
appData_df['AMT_CREDIT']=appData_df['AMT_CREDIT']/100000

bins = [0,1,2,3,4,5,6,7,8,9,10,100]
slots = ['0-1L','1L-2L', '2L-3L','3L-4L','4L-5L','5L-6L','6L-7L','7L-8L','8L-9L','9L-10L','10L Above']

appData_df['AMT_CREDIT_RANGE']=pd.cut(appData_df['AMT_CREDIT'],bins=bins,labels=slots)

In [188]:
appData_df["AMT_CREDIT_RANGE"].value_counts(normalize = True)*100

2L-3L        17.824728
10L Above    16.254703
5L-6L        11.131960
4L-5L        10.418489
1L-2L         9.801275
3L-4L         8.564897
6L-7L         7.820533
8L-9L         7.086576
7L-8L         6.241403
9L-10L        2.902986
0-1L          1.952450
Name: AMT_CREDIT_RANGE, dtype: float64

In [189]:
# Creating bins for AMT_GOODS_PRICE in term of Lakhs
appData_df['AMT_GOODS_PRICE']=appData_df['AMT_GOODS_PRICE']/100000

bins = [0,1,2,3,4,5,6,7,8,9,10,100]
slots = ['0-1L','1L-2L', '2L-3L','3L-4L','4L-5L','5L-6L','6L-7L','7L-8L','8L-9L','9L-10L','10L Above']

appData_df['AMT_GOODS_PRICE_RANGE']=pd.cut(appData_df['AMT_GOODS_PRICE'],bins=bins,labels=slots)

In [190]:
appData_df["AMT_GOODS_PRICE_RANGE"].value_counts(normalize = True)*100

2L-3L        20.427819
4L-5L        18.543906
6L-7L        13.027246
10L Above    11.105252
1L-2L        10.726712
8L-9L         6.992738
3L-4L         6.906485
5L-6L         4.269398
0-1L          2.834656
7L-8L         2.639690
9L-10L        2.526096
Name: AMT_GOODS_PRICE_RANGE, dtype: float64

In [ ]:
appData_df.head()

In [ ]:
appData_df.tail()